In [ ]:
import pandas as pd
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import svds
import pickle

In [ ]:
data_file = pd.read_table('data/training.csv', sep = ',', header=None, engine='python')

In [ ]:
data_file.describe
data_file.shape

In [ ]:
users = np.unique(data_file[0])
movies = np.unique(data_file[1])
 
number_of_rows = len(users)
number_of_columns = len(movies)

movie_indices, user_indices = {}, {}
 
for i in range(len(movies)):
    movie_indices[movies[i]] = i
    
for i in range(len(users)):
    user_indices[users[i]] = i

In [ ]:
np.unique(data_file[0])

In [ ]:
#scipy sparse matrix to store the 1M matrix
V = sp.lil_matrix((number_of_rows, number_of_columns))

#adds data into the sparse matrix
for line in data_file.values:
    u, i , r , gona = map(int,line)
    V[user_indices[u], movie_indices[i]] = r

In [ ]:
#as these operations consume a lot of time, it's better to save processed data 
with open('movielens_1M.pickle', 'wb') as handle:
    pickle.dump(V, handle)

In [ ]:
V.shape

In [ ]:
V.toarray()

In [ ]:
#as these operations consume a lot of time, it's better to save processed data 
#gets SVD components from 10M matrix
u,s, vt = svds(V, k = 32)
 
with open('movielens_1M_svd_u.pickle', 'wb') as handle:
    pickle.dump(u, handle)
with open('movielens_1M_svd_s.pickle', 'wb') as handle:
    pickle.dump(s, handle)
with open('movielens_1M_svd_vt.pickle', 'wb') as handle:
    pickle.dump(vt, handle)

In [ ]:
u.shape

In [ ]:
u

In [ ]:
s.shape

In [ ]:
s

In [ ]:
vt.shape

In [ ]:
vt

In [ ]:
negcounter = 0
poscounter = 0
for i in range(0,32):
    for factor in vt[i,:]:
        if factor > 0:
            poscounter = poscounter + 1
        else:
            negcounter = negcounter + 1
print(negcounter,poscounter)

In [ ]:
s_diag_matrix = np.zeros((s.shape[0], s.shape[0]))

for i in range(s.shape[0]):
    s_diag_matrix[i,i] = s[i]

In [ ]:
X_lr = np.dot(np.dot(u, s_diag_matrix), vt)

In [ ]:
X_lr

In [ ]:
X_lr.shape

In [ ]:
X_lr.size

In [ ]:
class Node:
    def __init__(self):
        self.itemFactors = np.empty(shape = (32,0))
        self.userFactors = np.empty(shape = (32,0))
        self.factor = None
        self.score = 0
        self.left = None
        self.right = None

In [ ]:
def printNode(node):
    print("itemFactors :") 
    #print(node.itemFactors)
    print(node.itemFactors.shape)
    print("userFactors :")
    #print(node.userFactors)
    print(node.userFactors.shape)
    print("factor :")
    print(node.factor)
    print("score :")
    print(node.score)
    #print("left :")
    #print(node.left)
    #print("right :")
    #print(node.right)
def printTree(node):
    printNode(node)
    if (node.left != None):
        print("LEFT :")
        printTree(node.left)
    if (node.right != None):
        print("RIGHT :")
        printTree(node.right)
def countLeaves(node):
    count = 0
    if (node.left != None):
        if (node.left.itemFactors.shape[1] <= 300):
            count += 1
        else:
            count += countLeaves(node.left)
    if (node.right != None):
        if (node.right.itemFactors.shape[1] <= 300):
            count += 1
        else:
            count += countLeaves(node.right)
    return count

In [ ]:
V = Node()
V.itemFactors = vt
V.userFactors = u

In [ ]:
V.itemFactors.shape

In [ ]:
V.userFactors.shape

In [ ]:
def splitNode(node, factor):
    node.left = Node()
    node.right = Node()
    left = 0 #flag and counter
    right = 0
    empt = Node()
    if (np.array_equal(node.itemFactors, empt.itemFactors) == False):   #bos item factor girebilir mi cond dene
    #if (len(node.itemFactors.shape) > 1):   #bos item factor girebilir mi cond dene
        #print(node.itemFactors.shape)
        for i in range(node.itemFactors.shape[1]):
            #print(node.itemFactors[:, i])
            if (node.itemFactors[:, i][factor] >= 0 ):
                if (left == 0):
                    node.left.itemFactors = node.itemFactors[:, i]
                else:
                    node.left.itemFactors = np.vstack((node.left.itemFactors, node.itemFactors[:, i]))
                left += 1
            else:
                #continue
                if (right == 0):
                    node.right.itemFactors = node.itemFactors[:, i]
                else:
                    node.right.itemFactors =np.vstack((node.right.itemFactors, node.itemFactors[:, i]))
                right += 1
        node.left.itemFactors = np.transpose(node.left.itemFactors)
        node.right.itemFactors = np.transpose(node.right.itemFactors)
    else:
        print('itemFactors not available')
    left = 0
    right = 0
    if (np.array_equal(node.userFactors, empt.userFactors) == False):   #bos user factor girebilme ihtimali dusun
    #if (len(node.userFactors.shape) > 1):   #bos user factor girebilme ihtimali dusun
        #print(node.userFactors.shape)
        for i in range(node.userFactors.shape[0]):
            if (node.userFactors[i, :][factor] >= 0 ):
                if (left == 0):
                    node.left.userFactors = node.userFactors[i, :]
                else:
                    node.left.userFactors = np.vstack((node.left.userFactors, node.userFactors[i, :]))
                left += 1
            else:
                if (right == 0):
                    node.right.userFactors = node.userFactors[i, :]
                else:
                    node.right.userFactors = np.vstack((node.right.userFactors, node.userFactors[i, :]))
                right += 1
    else:
        print('userFactors not available')
    #node.score =  computePrecision(node)

In [ ]:
def buildTree(node, factors):
    #printNode(node)
    #print(factors)
    empt = Node()
    if (len(node.itemFactors.shape) > 1 and node.itemFactors.shape[1] <= 300):
        print("Threshold value is reached")
        return node
    elif (len(node.itemFactors.shape) <= 1):
        print('one item factor')
        return node
    elif (node.itemFactors.shape[1] == 0):
        print('no item factor')
        return node
    winner = Node()
    if not factors:
        print('factors empty')
        return node
    for factor in factors:
        node.factor = factor
        splitNode(node,factor)
        if (node.score >= winner.score):
            winner = node
    factors.remove(winner.factor)
    node.factor = winner.factor
    lfactors = list(factors)
    rfactors = list(factors)
    buildTree(winner.left, lfactors)
    buildTree(winner.right, rfactors)
    return node
    

In [ ]:
print(V.itemFactors.shape)
print(V.userFactors.shape)

In [ ]:
splitNode(V,0)
print(V.right.itemFactors.shape)
print(V.left.itemFactors.shape)
print(V.right.userFactors.shape)
print(V.left.userFactors.shape)
printNode(V)

In [ ]:
factors = []
factors.extend(range(0,s.size))
buildTree(V,factors)

In [ ]:
print("ROOT :")
printTree(V)

In [ ]:
countLeaves(V)